# 3. doc2vec

In [15]:
from utils import load_data, dump_data
import pandas as pd
from sklearn.metrics import classification_report

VEC_SIZE = 300

dataset = load_data('cleaned')

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.33, random_state=42)

In [42]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit_transform(y_train)

y_train_enc = le.transform(y_train)
y_test_enc = le.transform(y_test)

In [43]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

X_doc_train = [TaggedDocument(words=words, tags=[str(i)])
               for i, words in enumerate(X_train)]

X_doc_test = [TaggedDocument(words=words, tags=[str(i)])
              for i, words in enumerate(X_test)]

model = Doc2Vec(dm=1, vector_size=VEC_SIZE, min_count=5, window=10, workers=4, epochs=100)
model.build_vocab(X_doc_train)
model.train(X_doc_train, total_examples=model.corpus_count, epochs=model.epochs)

In [44]:
import numpy as np

X_train_vec = np.array([model.infer_vector(doc.words) for doc in X_doc_train])
X_test_vec = np.array([model.infer_vector(doc.words) for doc in X_doc_test])

In [45]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=10, metric='cosine', n_jobs=-1)
neigh.fit(X_train_vec, y_train_enc)

y_pred = neigh.predict(X_test_vec)

In [46]:
pd.DataFrame(classification_report(y_test_enc, y_pred, output_dict=True))

/Users/h8si2w8/.pyenv/versions/3.8.2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,accuracy,macro avg,weighted avg
precision,0.069892,0.0,0.156909,0.063218,0.190476,0.041667,0.09375,0.100000,0.250,0.154362,...,0.454545,0.0,0.562500,0.425532,0.300000,0.333333,0.0,0.270985,0.209256,0.264544
recall,0.109244,0.0,0.296460,0.139241,0.166667,0.090909,0.09375,0.055556,0.375,0.225490,...,0.172414,0.0,0.107143,0.090909,0.048387,0.142857,0.0,0.270985,0.116543,0.270985
f1-score,0.085246,0.0,0.205207,0.086957,0.177778,0.057143,0.09375,0.071429,0.300,0.183267,...,0.250000,0.0,0.180000,0.149813,0.083333,0.200000,0.0,0.270985,0.131122,0.236911
support,119.000000,14.0,226.000000,79.000000,24.000000,22.000000,32.00000,18.000000,32.000,204.000000,...,29.000000,22.0,84.000000,220.000000,62.000000,21.000000,22.0,0.270985,20824.000000,20824.000000


**Далі:** [Використання довільних алгоритмів класифікації](04-vectors-algo.ipynb)